# YouTube Playlist Tools

Interactively export playlist metadata to CSV and download audio clips using `yt-dlp`. Channel URLs are supported; we fall back to each channel's uploads playlist automatically.

CSV columns: `url`, `title`, `description`, `duration_seconds`.


In [1]:
from __future__ import annotations

import sys
from pathlib import Path

PROJECT_ROOT = Path.cwd()
SRC_DIR = PROJECT_ROOT / "src"
if str(SRC_DIR) not in sys.path:
    sys.path.insert(0, str(SRC_DIR))


In [2]:
import subprocess

import ipywidgets as widgets
from IPython.display import display

from gigacan.playlist import export_playlist_metadata_csv


In [3]:
from pathlib import Path

playlist_url_widget = widgets.Text(
    value="",
    description="Playlist URL",
    placeholder="YouTube playlist or channel URL",
    layout=widgets.Layout(width="100%"),
)
playlist_output_path_widget = widgets.Text(
    value="metadata.csv",
    description="CSV Path",
    layout=widgets.Layout(width="100%"),
)
playlist_overwrite_widget = widgets.Checkbox(
    value=False,
    description="Overwrite existing file",
)
playlist_run_button = widgets.Button(
    description="Export Playlist CSV",
    button_style="primary",
    icon="save",
)
playlist_progress = widgets.IntProgress(
    value=0,
    min=0,
    max=1,
    description="Idle",
    bar_style="",
    layout=widgets.Layout(width="100%"),
)
playlist_status_label = widgets.HTML("<em>Waiting for input...</em>")
playlist_output = widgets.Output(
    layout=widgets.Layout(border="1px solid #ddd", padding="0.5rem")
)


def run_playlist_export(_):
    playlist_output.clear_output()
    playlist_progress.bar_style = ""
    playlist_progress.value = 0
    playlist_progress.max = 1
    playlist_progress.description = "Start"
    playlist_status_label.value = "<em>Fetching metadata...</em>"

    playlist_url = playlist_url_widget.value.strip()
    if not playlist_url:
        playlist_status_label.value = "<span style='color:#d33'>Provide a playlist or channel URL.</span>"
        with playlist_output:
            print("Please provide a playlist or channel URL.")
        return

    destination_text = playlist_output_path_widget.value.strip()
    destination = Path(destination_text) if destination_text else Path("metadata.csv")
    overwrite = playlist_overwrite_widget.value

    if destination.exists() and not overwrite:
        playlist_status_label.value = (
            f"<span style='color:#d33'>"
            f"Refusing to overwrite existing file {destination}."
            f" Enable overwrite to replace it.</span>"
        )
        with playlist_output:
            print("Refusing to overwrite existing file. Enable overwrite to replace it.")
        return

    try:
        destination.parent.mkdir(parents=True, exist_ok=True)
    except Exception as exc:
        playlist_status_label.value = (
            f"<span style='color:#d33'>Failed to prepare destination: {exc}</span>"
        )
        with playlist_output:
            print(f"Failed to create destination directory: {exc}")
        return

    def status_callback(message: str) -> None:
        playlist_status_label.value = message

    def progress_callback(done: int, total: int) -> None:
        if total:
            playlist_progress.max = total
            playlist_progress.value = done
            playlist_progress.description = f"{done}/{total}"
        else:
            playlist_progress.max = 1
            playlist_progress.value = 0
            playlist_progress.description = "Resolving..."

    try:
        count = export_playlist_metadata_csv(
            playlist_url,
            destination,
            progress_callback=progress_callback,
            status_callback=status_callback,
        )
    except Exception as exc:
        playlist_progress.bar_style = "danger"
        playlist_status_label.value = (
            f"<span style='color:#d33'>Failed: {exc}</span>"
        )
        with playlist_output:
            print(f"Failed to export playlist metadata: {exc}")
    else:
        playlist_progress.bar_style = "success"
        playlist_progress.value = playlist_progress.max
        playlist_progress.description = f"{playlist_progress.max}/{playlist_progress.max}"
        playlist_status_label.value = f"Finished {count} videos."
        with playlist_output:
            print(f"Wrote {count} rows to {destination.resolve()}")


playlist_run_button.on_click(run_playlist_export)

playlist_section = widgets.VBox(
    [
        widgets.HTML("""<h3>Playlist Metadata to CSV</h3><p>Fetch URL, title, description, and duration for each video in a playlist or channel.</p>"""),
        playlist_url_widget,
        playlist_output_path_widget,
        playlist_overwrite_widget,
        playlist_run_button,
        playlist_progress,
        playlist_status_label,
        playlist_output,
    ],
    layout=widgets.Layout(width="100%", gap="0.5rem"),
)


audio_url_widget = widgets.Text(
    value="",
    description="Video URL",
    placeholder="YouTube video, playlist, or channel URL",
    layout=widgets.Layout(width="100%"),
)
audio_output_dir_widget = widgets.Text(
    value="download",
    description="Output Dir",
    layout=widgets.Layout(width="100%"),
)
audio_format_widget = widgets.Dropdown(
    options=["opus", "mp3", "wav", "m4a"],
    value="opus",
    description="Format",
)
audio_sample_rate_widget = widgets.IntText(
    value=16000,
    description="Sample rate",
)
audio_run_button = widgets.Button(
    description="Download Audio",
    button_style="primary",
    icon="download",
)
audio_output = widgets.Output(
    layout=widgets.Layout(border="1px solid #ddd", padding="0.5rem")
)


def run_audio_download(_):
    audio_output.clear_output()
    with audio_output:
        url = audio_url_widget.value.strip()
        if not url:
            print("Please provide a video, playlist, or channel URL.")
            return
        output_dir_text = audio_output_dir_widget.value.strip() or "download"
        output_dir = Path(output_dir_text)
        audio_format = audio_format_widget.value
        sample_rate = audio_sample_rate_widget.value
        try:
            output_dir.mkdir(parents=True, exist_ok=True)
        except Exception as exc:
            print(f"Failed to create output directory {output_dir}: {exc}")
            return
        command = [
            "yt-dlp",
            "-P", str(output_dir),
            "-x",
            "--audio-format", audio_format,
            "--yes-playlist",
        ]
        if sample_rate:
            command.extend(["--postprocessor-args", f"-ar {int(sample_rate)}"])
        command.append(url)
        try:
            subprocess.run(command, check=True)
        except FileNotFoundError:
            print("yt-dlp not found. Install it with `pip install yt-dlp`.")
        except subprocess.CalledProcessError as exc:
            print(f"yt-dlp exited with code {exc.returncode}.")
        else:
            print(f"Download complete. Files saved in {output_dir.resolve()}")


audio_run_button.on_click(run_audio_download)

audio_section = widgets.VBox(
    [
        widgets.HTML("""<h3>Download Audio</h3><p>Extract audio using yt-dlp with your preferred format.</p>"""),
        audio_url_widget,
        audio_output_dir_widget,
        audio_format_widget,
        audio_sample_rate_widget,
        audio_run_button,
        audio_output,
    ],
    layout=widgets.Layout(width="100%", gap="0.5rem"),
)


interface = widgets.Tab(children=[playlist_section, audio_section])
interface.set_title(0, "Playlist CSV")
interface.set_title(1, "Download Audio")

display(interface)
